# Training Deep Neural Networks

## Practical 2

_**Experiment on Fashion MNIST or any other appropriate dataset to check if reusing pretrained layers in transfer learning makes the training possible with less data and saves training time. Also check for any model performance improvement.**_

First a model will be trained on set A (for classification task with 8 classes such as trousers, dresses, coats, sandals, shirts, sneakers, bags, and ankle boots in Fashion MNIST dataset). Then the learning will be reused for another a binary classification task on set B (the remaining 2 classes such as T-shirts/tops and pullovers in the saame dataset) since classes in set A are somewhat similar to classes in set B.

In [40]:
# Imports required packages

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

In [19]:
# Loads fashion mnist dataset
fashion = fashion_mnist.load_data()

In [25]:
# Considering dataset is organized in tuple, items are referenced as follows
(X_train_full, y_train_full), (X_test, y_test) = fashion

In [27]:
# Checks the shape of the datasets
print("Train dataset shape:", X_train_full.shape,
      "\nTest dataset shape:", X_test.shape)

Train dataset shape: (60000, 28, 28) 
Test dataset shape: (10000, 28, 28)


In [36]:
# Splits train dataset further to seperate 5000 instances to be used as validation set

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=5000, random_state=42, stratify=y_train_full)

In [4]:
# Each training and test example is assigned to one of the following labels.
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", \
               "Shirt", "Sneaker", "Bag", "Ankle boot"]

**Splits Fashion MNIST into tasks A and B, then train and save.**

In [7]:
pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

In [9]:
def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

In [42]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_val_A, y_val_A), (X_val_B, y_val_B) = split_dataset(X_val, y_val)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [44]:
tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

2024-08-28 09:31:08.660696: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [46]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_val_A, y_val_A))

Epoch 1/20
1375/1375 [==============================] - 10s 7ms/step - loss: 5.6492 - accuracy: 0.4103 - val_loss: 1.2615 - val_accuracy: 0.5045
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 1.1841 - accuracy: 0.5474 - val_loss: 1.0626 - val_accuracy: 0.5690
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 1.0337 - accuracy: 0.5819 - val_loss: 0.9390 - val_accuracy: 0.5913
Epoch 4/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.8505 - accuracy: 0.6569 - val_loss: 0.7599 - val_accuracy: 0.7048
Epoch 5/20
1375/1375 [==============================] - 10s 7ms/step - loss: 0.7295 - accuracy: 0.7153 - val_loss: 0.6834 - val_accuracy: 0.7232
Epoch 6/20
1375/1375 [==============================] - 9s 6ms/step - loss: 0.6672 - accuracy: 0.7342 - val_loss: 0.6690 - val_accuracy: 0.7305
Epoch 7/20
1375/1375 [==============================] - 14s 10ms/step - loss: 0.6254 - accuracy: 0.7438 - val_loss: 0.6013 - val_accur

In [48]:
model_A.save("./models/my_fashion_mnist_model_A")

INFO:tensorflow:Assets written to: ./my_fashion_mnist_model_A/assets


INFO:tensorflow:Assets written to: ./my_fashion_mnist_model_A/assets


Now, to realize whether if transfer learning works or not, first train model B, then evaluate it, without reusing model A

In [51]:
tf.random.set_seed(42)

model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_val_B, y_val_B))

Epoch 1/20
7/7 [==============================] - 1s 47ms/step - loss: 241.5345 - accuracy: 0.5050 - val_loss: 13.2833 - val_accuracy: 0.5050
Epoch 2/20
7/7 [==============================] - 0s 65ms/step - loss: 6.2211 - accuracy: 0.6350 - val_loss: 0.5385 - val_accuracy: 0.9120
Epoch 3/20
7/7 [==============================] - 0s 36ms/step - loss: 0.3296 - accuracy: 0.9300 - val_loss: 0.4761 - val_accuracy: 0.9040
Epoch 4/20
7/7 [==============================] - 0s 29ms/step - loss: 0.2511 - accuracy: 0.9200 - val_loss: 0.4628 - val_accuracy: 0.9200
Epoch 5/20
7/7 [==============================] - 0s 26ms/step - loss: 0.1944 - accuracy: 0.9250 - val_loss: 0.3994 - val_accuracy: 0.9200
Epoch 6/20
7/7 [==============================] - 0s 50ms/step - loss: 0.1350 - accuracy: 0.9350 - val_loss: 0.3874 - val_accuracy: 0.9200
Epoch 7/20
7/7 [==============================] - 0s 42ms/step - loss: 0.1296 - accuracy: 0.9450 - val_loss: 0.3898 - val_accuracy: 0.9180
Epoch 8/20
7/7 [========

In [53]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.3673 - accuracy: 0.9180


[0.36726489663124084, 0.9179999828338623]

Model B reaches 91.79% accuracy on the test set. Now let's try reusing the pretrained model A.

In [56]:
model_A = tf.keras.models.load_model("my_fashion_mnist_model_A")

model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])

model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

As model_B_on_A and model_A actually share layers now, training one will update both models. To avoid that model_B_on_A should be built on top of a clone of model_A.

In [59]:
tf.random.set_seed(42)

model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [61]:
# Creates model_B_on_A just like in the previous cell

model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])

model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [63]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [65]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_val_B, y_val_B))

Epoch 1/4
7/7 [==============================] - 1s 52ms/step - loss: 0.6611 - accuracy: 0.6550 - val_loss: 0.6802 - val_accuracy: 0.6200
Epoch 2/4
7/7 [==============================] - 0s 28ms/step - loss: 0.6602 - accuracy: 0.6550 - val_loss: 0.6796 - val_accuracy: 0.6200
Epoch 3/4
7/7 [==============================] - 0s 46ms/step - loss: 0.6593 - accuracy: 0.6550 - val_loss: 0.6789 - val_accuracy: 0.6200
Epoch 4/4
7/7 [==============================] - 1s 84ms/step - loss: 0.6582 - accuracy: 0.6550 - val_loss: 0.6784 - val_accuracy: 0.6200


In [67]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_val_B, y_val_B))

Epoch 1/16
7/7 [==============================] - 1s 86ms/step - loss: 0.6513 - accuracy: 0.6650 - val_loss: 0.6738 - val_accuracy: 0.6730
Epoch 2/16
7/7 [==============================] - 0s 25ms/step - loss: 0.6319 - accuracy: 0.7200 - val_loss: 0.6642 - val_accuracy: 0.6050
Epoch 3/16
7/7 [==============================] - 0s 22ms/step - loss: 0.6326 - accuracy: 0.6650 - val_loss: 0.6539 - val_accuracy: 0.6500
Epoch 4/16
7/7 [==============================] - 0s 25ms/step - loss: 0.5895 - accuracy: 0.7100 - val_loss: 0.6334 - val_accuracy: 0.7010
Epoch 5/16
7/7 [==============================] - 1s 85ms/step - loss: 0.5804 - accuracy: 0.7400 - val_loss: 0.6173 - val_accuracy: 0.7160
Epoch 6/16
7/7 [==============================] - 0s 19ms/step - loss: 0.5460 - accuracy: 0.7900 - val_loss: 0.5787 - val_accuracy: 0.7940
Epoch 7/16
7/7 [==============================] - 0s 44ms/step - loss: 0.5392 - accuracy: 0.8000 - val_loss: 0.5947 - val_accuracy: 0.7130
Epoch 8/16
7/7 [===========

In [69]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 6ms/step - loss: 0.4682 - accuracy: 0.8745


[0.4681757092475891, 0.8744999766349792]

TO-DO:
- model's accuracy improvement needs to be calculated
- Drop in error rate (in percentile) needs to be calculated.